El objetivo es encotrar la informacion mas posible, y que sea de calidad. De acuerdo a los criterios de PLD

| Información requerida                   | Información obtenida                              |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 | MECA.CAPF_CLIENTES_POR_AREA relacionar con esta tabla |
| nombre cliente o razón social           | CLI_NOMBRE                                       |
| número cliente                          | CLI_CLAVE                                        |
| estatus                                 | CLI_EST_CLAVE                                    |
| número contrato                         |                                                  |
| género                                  |                                                  |
| fecha_nacimiento o de constitución      |                                                  |
| entidad federativa nacimiento           |                                                  |
| país de nacimiento                      |                                                  |
| nacionalidad                            | CLI_NACIONALIDAD                                 |
| profesión                               |                                                  |
| calle                                   | CLI_DIRECCION                                    |
| número_exterior                         |                                                  |
| número_interior                         |                                                  |
| colonia_urbanización                    |                                                  |
| delegación_municipio                    |                                                  |
| ciudad_población                        |                                                  |
| entidad_federativa                      | CLI_EDO_CLAVE                                    |
| código postal                           |                                                  |
| país                                    | CLI_EDO_PAI_CLAVE                                |
| teléfono                                | CLI_TELEFONO                                     |
| correo_electrónico                      |                                                  |
| rfc                                     | CLI_RFC_LETRAS, CLI_RFC_FECHA, CLI_RFC_HOMO      |
| curp                                    |                                                  |
| fiel                                    |                                                  |
| representante legal                     |                                                  |
| tipo de persona                         | CLI_TDC_CLAVE                                    |
| clasificación por grado de riesgo       |                                                  |
| pep (persona políticamente expuesta)    |                                                  |
| razón                                   |                                                  |
| producto contratado                     | por movimientos y operaciones                    |
| cambios en perfil transaccional         |                                                  |

In [2]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from datetime import datetime
import cx_Oracle
import pandas as pd
import os

In [87]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Transformacion_clientes_MECA.dat",low_memory=False)

## NEGOCIO
SE ENCONTRO UNA POSIBLE RELACION CON RESPECTO A NEGOCIO POR CATALOGO DE CLIENTE_POR_AREA TODOS RELACIONADOS A UNIDAD DE MERCADO DE CAPITALES, SE HACE UNA ESTANDARIZACION POR MAYUSCULA Y MAPEANDO VALORES NULOS A NO SE ENCONTRO INFORMACION

In [43]:
df.value_counts("DESCRIPCIONES")

DESCRIPCIONES
UNIDAD DE MERCADO DE CAPITALES    49719
Name: count, dtype: int64

In [48]:
df = df.rename(columns={"DESCRIPCIONES": "NEGOCIO"})
df["NEGOCIO"] = df["NEGOCIO"].apply(lambda x: x.upper() if pd.notna(x) else "NO SE ENCONTRO INFORMACION")

In [49]:
df.value_counts("NEGOCIO")

NEGOCIO
UNIDAD DE MERCADO DE CAPITALES    49719
Name: count, dtype: int64

## NOMBRE CLIENTE O RAZON SOCIAL

SE NORMALIZARA PASANDO A MAYUSCULA, DADO POR LA COMPLEJIDAD DEL NOMBRE QUE PUEDE CONTENER CARACTERES ESPECIALES SE CONSERVA EL NOMBRE COMO VIENE DE LA BASE DE DATOS, SOLO SE QUITARA VALORES ATIPICOS (OUTLIERS)


In [53]:
# Supongamos que df es tu DataFrame original
df_conteo = df[["CLI_CLAVE", "CLI_NOMBRE"]]
df_conteo["SIZE_NAME"] = df_conteo["CLI_NOMBRE"].apply(lambda x: len(x))

# Calcular los cuartiles y el rango intercuartílico (IQR)
Q1 = df_conteo["SIZE_NAME"].quantile(0.25)
Q3 = df_conteo["SIZE_NAME"].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites para los valores atípicos
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar los valores atípicos
valores_atipicos = df_conteo[(df_conteo["SIZE_NAME"] < limite_inferior) | (df_conteo["SIZE_NAME"] > limite_superior)]

print(valores_atipicos)

      CLI_CLAVE                                         CLI_NOMBRE  SIZE_NAME
12       INGBAR  ING (MEXICO), S.A.DE C.V. CASA DE BOLSA, ING G...         61
13       INGBAR  ING (MEXICO), S.A.DE C.V. CASA DE BOLSA, ING G...         61
14       INGBAR  ING (MEXICO), S.A.DE C.V. CASA DE BOLSA, ING G...         61
15       INGBAR  ING (MEXICO), S.A.DE C.V. CASA DE BOLSA, ING G...         61
16       INGBAR  ING (MEXICO), S.A.DE C.V. CASA DE BOLSA, ING G...         61
...         ...                                                ...        ...
49602    MULTIV  CASA DE BOLSA MULTIVA, S.A. DE C.V. GRUPO FINA...         60
49603    MULTIV  CASA DE BOLSA MULTIVA, S.A. DE C.V. GRUPO FINA...         60
49608    MULTIV  CASA DE BOLSA MULTIVA, S.A. DE C.V. GRUPO FINA...         60
49609    MULTIV  CASA DE BOLSA MULTIVA, S.A. DE C.V. GRUPO FINA...         60
49610    MULTIV  CASA DE BOLSA MULTIVA, S.A. DE C.V. GRUPO FINA...         60

[3791 rows x 3 columns]


/tmp/ipykernel_3183/64129732.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conteo["SIZE_NAME"] = df_conteo["CLI_NOMBRE"].apply(lambda x: len(x))


In [54]:
df["CLI_NOMBRE"] = df["CLI_NOMBRE"].apply(lambda x: x.upper())
df = df.rename(columns={"CLI_NOMBRE":"nombre cliente o razón social".upper()})

In [109]:
df.value_counts("CLI_NOMBRE")

CLI_NOMBRE
OPERACION INTERNA (CARTERAS CAPITALES)                           21437
IXE CASA DE BOLSA, S.A. DE C.V.                                   7551
ACTINVER CASA DE BOLSA                                            3926
CASA DE BOLSA MULTIVA, S.A. DE C.V. GRUPO FINANCIERO MULTIVA      3539
VECTOR, CASA DE BOLSA, S.A DECV                                   2679
CASA DE BOLSA BANORTE, S.A. DE C.V                                2531
CASA DE BOLSA SANTANDER MEXICO, S.A. DE C.V.                      2006
VALORES FINAMEX, S.A. DE C.V.                                     1994
ACCIONES Y VALORES DE MEXICO                                       808
SCOTIA INVERLAT CASA DE BOLSA, S.A. DE C.V.                        672
MERRILL LYNCH MEXICO, S.A. DE C.V. CASA DE BOLSA                   616
FIDEICOMISOS NAFTRACK                                              599
INTERACCIONES, CASA DE BOLSA, SA                                   464
GRUPO BURSATIL MEXICANO, S.A. DE C.V.                             

In [111]:
df.value_counts("CMV_DESCRIPCION")

CMV_DESCRIPCION
COMPRA                            9455
COMISION DE COMPRA                9454
IMPUESTO DE COMPRA                9454
VENTA CON UTILIDAD                4765
UTILIDAD                          4756
COMSION DE VENTA CON UTILIDAD     4755
IMPUESTO DE VENTA CON UTILIDAD    4755
PERDIDA                            550
COMISION DE VENTA CON PERDIDA      549
IMPUESTO DE VENTA CON PERDIDA      549
VENTA CON PERDIDA                  540
LIQUIDACION COMPRA                  51
LIQUIDACION VENTA CON PERDIDA       49
LIQUIDACION VENTA CON UTILIDAD      37
Name: count, dtype: int64

## ESTATUS
SE MAPEARA A VIGENTE O NO VIGENTE DE LA COLUMNA CLI_EST_CLAVE

In [55]:
df.value_counts("CLI_EST_CLAVE")

CLI_EST_CLAVE
VIG    49719
Name: count, dtype: int64

In [56]:
df = df.rename(columns={"CLI_EST_CLAVE":"ESTATUS"})
df["ESTATUS"] = df["ESTATUS"].apply(lambda x: "VIGENTE" if x == "VIG" else "NO VIGENTE")

## NACIONALIDAD 
SE MAPEO MEX: MEXICANA, EXT: EXTRANJERA, EN CASO DE SER NULO SE "NO SE ENCONTRO INFORMACION" 

In [ ]:
df.value_counts("CLI_NACIONALIDAD")

In [ ]:
df = df.rename(columns={"CLI_NACIONALIDAD": "NACIONALIDAD"})
df["NACIONALIDAD"] = df["NACIONALIDAD"].apply(lambda x: "NO SE ENCONTRO INFORMACION" if pd.isna(x) else x)
df["NACIONALIDAD"] = df["NACIONALIDAD"].apply(lambda x: "MEXICANA" if x == "MEX" else x)
df["NACIONALIDAD"] = df["NACIONALIDAD"].apply(lambda x: "EXTRANJERA" if x == "EXT" else x)

df.value_counts("NACIONALIDAD")

## DIRECCION
ESTE CAMPO SE DISPERSARA COMO CALLE

In [108]:
len(df["CLI_NOMBRE"].unique())

23

In [68]:
palabras_no_validas = ["PENDIENTE", "JHGJHG","P"]

In [88]:
df.value_counts("CLI_DIRECCION").index

Index(['P',
       'BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.',
       'NO SE TIENE REGISTRADA LA INFORMACION',
       'RIO AMAZONAS #91 COL. CUAUHTEMOC CP 06500',
       'REFORMA #398 COL. JUAREZ PB CP 06600',
       'BOSQUE DE CIRUELOS 120, BOSQUES DE LAS LOMAS C.P. 11700, MEXICO, D.F.',
       'PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEMOC CP 06500',
       'INSURGENTES SUR 1886 COL.FLORIDA', 'JHGJHG', 'PENDIENTE',
       'MONTES URALES #424 PISO 3, COLONIA LOMAS DE CHAPULTEPEC C.P. 11000',
       'INSURGENTES SUR 1971'],
      dtype='object', name='CLI_DIRECCION')

In [72]:
df["CLI_DIRECCION"] = df["CLI_DIRECCION"].apply(lambda x: x.upper() if x not in palabras_no_validas else "NO SE TIENE REGISTRADA LA INFORMACION")

In [ ]:
df.value_counts("CLI_DIRECCION")

CLI_DIRECCION
NO SE TIENE REGISTRADA LA INFORMACION                                                      41757
BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.     3926
RIO AMAZONAS #91 COL. CUAUHTEMOC CP 06500                                                   1994
REFORMA #398 COL. JUAREZ PB CP 06600                                                         808
BOSQUE DE CIRUELOS 120, BOSQUES DE LAS LOMAS C.P. 11700, MEXICO, D.F.                        672
PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEMOC CP 06500                                    464
INSURGENTES SUR 1886 COL.FLORIDA                                                              91
MONTES URALES #424 PISO 3, COLONIA LOMAS DE CHAPULTEPEC C.P. 11000                             4
INSURGENTES SUR 1971                                                                           3
Name: count, dtype: int64

In [ ]:
def separar_direccion(direccion):
    # Ejemplo simple de separación usando regex
    match = re.match(r'^(.*?)(\d+.*?)(COL\..*?)(CP \d+)(.*)$', direccion)
    if match:
        return match.groups()
    else:
        return (direccion, "", "", "", "")

In [75]:
separar_direccion("BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.")

('BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.',
 '',
 '',
 '',
 '')

In [79]:
sep = "BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.".split(",")
sep = [s.split("COL") if "COL" in s else s for s in sep]

In [95]:
df[df["CLI_EDO_CLAVE"].notna()]

,DESCRIPCIONES,CLI_CLAVE,CLI_NOMBRE,CLI_EST_CLAVE,CLI_NACIONALIDAD,CLI_DIRECCION,CLI_EDO_CLAVE,CLI_EDO_PAI_CLAVE,CLI_TELEFONO,CLI_RFC_LETRAS,...,CMV_DESCRIPCION,MPA_DESCRIPCION,OPE_INS_CLAVE,OPE_ESTATUS,OPE_FECHA_VENCIMIENTO,OPE_TIPO_PRESTAMO,OPE_TASA_PRESTAMO,OPE_PRECIO_PRESTAMO,OPE_PRECIO_PACTADA,INS_DESCRIPCION
1224,UNIDAD DE MERCADO DE CAPITALES,INTER,"INTERACCIONES, CASA DE BOLSA, SA",VIG,MEX,PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEM...,MDF,MEX,2-08-30-98 CARLOS TAPIA,ICB,...,COMPRA,LINEA BANXICO (SIACC),ACC,LIQ,NaN,ORD,NaN,34.972676,NaN,ACCIONES
1225,UNIDAD DE MERCADO DE CAPITALES,INTER,"INTERACCIONES, CASA DE BOLSA, SA",VIG,MEX,PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEM...,MDF,MEX,2-08-30-98 CARLOS TAPIA,ICB,...,COMISION DE COMPRA,LINEA BANXICO (SIACC),ACC,LIQ,NaN,ORD,NaN,34.972676,NaN,ACCIONES
1226,UNIDAD DE MERCADO DE CAPITALES,INTER,"INTERACCIONES, CASA DE BOLSA, SA",VIG,MEX,PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEM...,MDF,MEX,2-08-30-98 CARLOS TAPIA,ICB,...,IMPUESTO DE COMPRA,LINEA BANXICO (SIACC),ACC,LIQ,NaN,ORD,NaN,34.972676,NaN,ACCIONES
1597,UNIDAD DE MERCADO DE CAPITALES,BETA,BETA CAPITAL MANAGMENT,VIG,EXT,P,N.Y,USA,NaN,XXX,...,COMPRA,LINEA OPERACIONES USD,ACC,LIQ,NaN,ORD,NaN,38.458675,NaN,ACCIONES
1598,UNIDAD DE MERCADO DE CAPITALES,BETA,BETA CAPITAL MANAGMENT,VIG,EXT,P,N.Y,USA,NaN,XXX,...,COMISION DE COMPRA,LINEA OPERACIONES USD,ACC,LIQ,NaN,ORD,NaN,38.446615,NaN,ACCIONES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40998,UNIDAD DE MERCADO DE CAPITALES,BETA,BETA CAPITAL MANAGMENT,VIG,EXT,P,N.Y,USA,NaN,XXX,...,LIQUIDACION COMPRA,LINEA OPERACIONES USD,ACC,LIQ,NaN,ORD,NaN,NaN,NaN,ACCIONES
49259,UNIDAD DE MERCADO DE CAPITALES,OPIUSD,OPERACION INTERNA ADS'S,VIG,EXT,JHGJHG,N.Y,USA,5654,1,...,COMPRA,LINEA OPERACIONES USD,ACC,LIQ,NaN,ORD,NaN,NaN,NaN,ACCIONES
49260,UNIDAD DE MERCADO DE CAPITALES,OPIUSD,OPERACION INTERNA ADS'S,VIG,EXT,JHGJHG,N.Y,USA,5654,1,...,COMISION DE COMPRA,LINEA OPERACIONES USD,ACC,CAN,NaN,ORD,NaN,NaN,NaN,ACCIONES
49261,UNIDAD DE MERCADO DE CAPITALES,OPIUSD,OPERACION INTERNA ADS'S,VIG,EXT,JHGJHG,N.Y,USA,5654,1,...,IMPUESTO DE COMPRA,LINEA OPERACIONES USD,ACC,LIQ,NaN,ORD,NaN,71.465993,71.465993,ACCIONES


In [94]:
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer

# Datos de ejemplo
data = pd.DataFrame({
    'CLI_NOMBRE': ['P',
       'BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.',
       'NO SE TIENE REGISTRADA LA INFORMACION',
       'RIO AMAZONAS #91 COL. CUAUHTEMOC CP 06500',
       'REFORMA #398 COL. JUAREZ PB CP 06600',
       'BOSQUE DE CIRUELOS 120, BOSQUES DE LAS LOMAS C.P. 11700, MEXICO, D.F.',
       'PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEMOC CP 06500',
       'INSURGENTES SUR 1886 COL.FLORIDA', 'JHGJHG', 'PENDIENTE',
       'MONTES URALES #424 PISO 3, COLONIA LOMAS DE CHAPULTEPEC C.P. 11000',
       'INSURGENTES SUR 1971']
})

# Vectorización TF-IDF
vectorizer = TfidfVectorizer()
encoded_data = vectorizer.fit_transform(data['CLI_NOMBRE']).toarray()

# Aplicar IsolationForest
model = IsolationForest(contamination=0.001)
model.fit(encoded_data)
outliers = model.predict(encoded_data)

# Identificar valores atípicos
data['ANOMALY'] = outliers
print(data[data['ANOMALY'] == -1])

                                          CLI_NOMBRE  ANOMALY
1  BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES P...       -1


In [91]:
data

,CLI_NOMBRE,ANOMALY
0,P,1
1,BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES P...,-1
2,NO SE TIENE REGISTRADA LA INFORMACION,1
3,RIO AMAZONAS #91 COL. CUAUHTEMOC CP 06500,1
4,REFORMA #398 COL. JUAREZ PB CP 06600,1
5,"BOSQUE DE CIRUELOS 120, BOSQUES DE LAS LOMAS C...",-1
6,PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEM...,1
7,INSURGENTES SUR 1886 COL.FLORIDA,1
8,JHGJHG,1
9,PENDIENTE,1


In [90]:
encoded_data

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.27285473, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.27285473,
        0.27285473, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.27285473, 0.        , 0. 

## CLI_TELEFONO

In [104]:
df["CLI_NOMBRE"].unique()

array(['VECTOR, CASA DE BOLSA, S.A DECV',
       'CASA DE BOLSA BANORTE, S.A. DE C.V',
       'ING (MEXICO), S.A.DE C.V. CASA DE BOLSA, ING GRUPO FINANCIERO',
       'CASA DE BOLSA MULTIVA, S.A. DE C.V. GRUPO FINANCIERO MULTIVA',
       'OPERACION INTERNA (CARTERAS CAPITALES)',
       'VALUE CASA DE BOLSA, S.A. DE C.V',
       'SCOTIA INVERLAT CASA DE BOLSA, S.A. DE C.V.',
       'GRUPO BURSATIL MEXICANO, S.A. DE C.V.',
       'IXE CASA DE BOLSA, S.A. DE C.V.',
       'CASA DE BOLSA SANTANDER MEXICO, S.A. DE C.V.',
       'ACCIONES Y VALORES DE MEXICO', 'ACTINVER CASA DE BOLSA',
       'INTERACCIONES, CASA DE BOLSA, SA', 'BETA CAPITAL MANAGMENT',
       'MONEX CASA DE BOLSA, S.A. DE C.V.', "OPERACION INTERNA ADS'S",
       'VALORES FINAMEX, S.A. DE C.V.',
       'CASA DE BOLSA BBVA BANCOMER, S.A. DE C.V.',
       'FIDEICOMISOS NAFTRACK', 'INVEX, CASA DE BOLSA, S.A. DE C.V',
       'NACIONAL FINANCIERA, S.N.C.', 'INVERSORA BURSATIL, S.A. DE C.V',
       'MERRILL LYNCH MEXICO, S.A. DE C.

In [106]:
df_duplicate = df[df["CLI_TELEFONO"].notna()].drop_duplicates("CLI_NOMBRE")

In [ ]:
df_duplicate

## RFC
ESTANDARIZACION DEL RFC GENERICO XXX-XXXXXX-XXX Y 
CON MAYUSCULAS Y CONCATENACION

In [112]:
# GUARDAMOS LAS COLUMANS
columnas =  df.columns

In [113]:
df["CLI_RFC_LETRAS"]= df["CLI_RFC_LETRAS"].apply(lambda x: x.upper() if pd.notna(x) else "XXX")

In [114]:
df["CLI_RFC_FECHA"] = df["CLI_RFC_FECHA"].apply(lambda x: int(x) if pd.notna(x) else "XXXXXX") 

In [116]:
df["CLI_RFC_HOMO"] = df["CLI_RFC_HOMO"].apply(lambda x: x.upper() if pd.notna(x) else "XXX") 

In [117]:
df["RFC"] = df.apply(lambda row: f'{row["CLI_RFC_LETRAS"]}{row["CLI_RFC_FECHA"]}{row["CLI_RFC_HOMO"]}',axis=1)

In [118]:
new_columnas = [col for col in columnas if col not in ["CLI_RFC_LETRAS","CLI_RFC_FECHA","CLI_RFC_HOMO"]]

In [119]:
df = df[new_columnas + ["RFC"]]

In [120]:
df.value_counts("RFC")

RFC
XXXXXXXXXXXX    22036
XXX123456XXX    20399
ACB7609076M2     3926
VFI770713PB8     1994
AVM760909UK0      808
ICB8704225X7      464
111                85
CBB960628JY9        4
NFI3406305TO        3
Name: count, dtype: int64